In [2]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.0.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.6
✔ tidyr   0.8.1     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [3]:
df <- read_csv("LigandEfficiencyRawData.csv", col_names=TRUE)

Parsed with column specification:
cols(
  Ligands = col_character(),
  dG = col_double(),
  MW = col_double(),
  TPSA = col_double(),
  Isoform = col_integer()
)


In [4]:
Isoform <- df$Isoform
Ligand <- df$Ligands
TPSA <- df$TPSA
MW <- df$MW
deltaG <- df$dG

$$ K_{i}(\mu M) = exp\left(\frac{\Delta G (kcal/mol) \times 1000}{1.986 \times 298.15} \right) \times \frac{1}{10^{-6}} $$
$$ pK_{i} = -log_{10}\left(K_{i}(mole/L)\right)$$
$$ Ligand Efficiency = -\frac{\Delta G}{HAC}$$
$$ B.E.I. = \frac{pK_{i}}{MW (kDa)} $$
$$ S.E.I. = \frac{pK_{i}}{(TPSA/100)} $$

In [6]:
# Ki in micromolars
Ki <- function(dG)
{
    return(exp((dG*1000)/(1.986*298.15))*(1/(10**(-6))))
}

# pKi
pKi <- function(Ki)
{
    # Converting Ki from uM/L
    # to M/L
    return(-log10(Ki/1000000))
}

# Ligand Efficiency
LE <- function()
{
    return(-(deltaG/HAC))
}

# Binding Efficiency Index
BEI <- function()
{
    # Convert MW to kDa
    kDa <- MW/1000
    return(pki/kDa)
}

# Surface Binding Efficiency Index
SEI <- function()
{
    # normalisze
    norm <- TPSA/100
    return(pki/norm)
}

In [7]:
ki <- round(Ki(deltaG), digits=3)

In [8]:
pki <- round(pKi(ki), digits=3)

In [9]:
pki %>% head

[1] 5.281 5.281 5.281 5.134 5.134 5.134

In [12]:
bei <- round(BEI(), digits=3)
sei <- round(SEI(), digits=3)

In [13]:
ki %>% head

[1] 5.238 5.238 5.238 7.343 7.343 7.343

In [14]:
pki %>% head

[1] 5.281 5.281 5.281 5.134 5.134 5.134

In [15]:
bei %>% head

[1] 18.836 19.251 19.324 18.785 18.853 19.646

In [16]:
sei %>% head

[1] 6.124 8.006 6.091 6.950 8.347 6.260

In [18]:
bindTable <- cbind(Ligand, Isoform, deltaG,MW,ki,pki,bei,sei)

In [19]:
colnames(bindTable) <- c("Ligand", "Isoform", "deltaG", "MW", "Ki", "pKi", "BEI", "SEI")

In [20]:
df <- data.frame(bindTable)

In [21]:
df %>% head

Ligand,Isoform,deltaG,MW,Ki,pKi,BEI,SEI
reduce-134,1,-7.2,280.369,5.238,5.281,18.836,6.124
reduce-449,1,-7.2,274.318,5.238,5.281,19.251,8.006
reduce-439,1,-7.2,273.291,5.238,5.281,19.324,6.091
reduce-419,1,-7,273.31,7.343,5.134,18.785,6.95
reduce-83,1,-7,272.322,7.343,5.134,18.853,8.347
reduce-287,1,-7,261.32,7.343,5.134,19.646,6.26


In [22]:
write_csv(df, path = "screenlib-bei-sei.csv", col_names=TRUE)

In [68]:
sessionInfo()

R version 3.5.1 (2018-07-02)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Ubuntu 18.04.1 LTS

Matrix products: default
BLAS: /usr/lib/x86_64-linux-gnu/openblas/libblas.so.3
LAPACK: /usr/lib/x86_64-linux-gnu/libopenblasp-r0.2.20.so

locale:
 [1] LC_CTYPE=en_IN       LC_NUMERIC=C         LC_TIME=en_IN       
 [4] LC_COLLATE=en_IN     LC_MONETARY=en_IN    LC_MESSAGES=en_IN   
 [7] LC_PAPER=en_IN       LC_NAME=C            LC_ADDRESS=C        
[10] LC_TELEPHONE=C       LC_MEASUREMENT=en_IN LC_IDENTIFICATION=C 

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] bindrcpp_0.2.2  forcats_0.3.0   stringr_1.3.1   dplyr_0.7.6    
 [5] purrr_0.2.5     readr_1.1.1     tidyr_0.8.1     tibble_1.4.2   
 [9] ggplot2_3.0.0   tidyverse_1.2.1

loaded via a namespace (and not attached):
 [1] pbdZMQ_0.3-3         tidyselect_0.2.4     repr_0.15.0         
 [4] haven_1.1.2          lattice_0.20-35      colorspace_1.3-2    